# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b']['call'])

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

**#1. What test is appropriate for this problem? Does CLT apply?**

The best test for this problem would be a two-tailed permutation test since our population standard deviation is unknown and our sample size is large (n > 30). The central limit theory does apply. 

**#2. What are the null and alternate hypotheses?**

The null hypothesis is that race has no impact on the amount of callbacks for resumes

The alternatice hypothesis is that race affects the amount of callbacks for resumes

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [21]:
# Your solution to Q3 here
#Compute margin of error, confidence interval, and p-value. 
#Try using both the bootstrapping and the frequentist statistical approaches.

# Amount of white and black resumes
n_w = len(w)
n_b = len(b)

# Data on whites and blacks that were called back
w_c = w['call']
b_c = b['call']

# frequentist 
t_stat, p = stats.ttest_ind(w_c , b_c, equal_var = False)
print('t-statistic =', t_stat)
print('p-value =', p)

# Standard Error
SE = np.sqrt(w_c.var() / n_w + b_c.var() / n_b)

# Margin of Error
MOE = 1.96 * SE
print('Margin of Error =', MOE)

# Difference of means
diff_means = np.abs(np.mean(w_c) - np.mean(b_c))

# Confidence Interval
confidence_interval = [diff_means - MOE, diff_means + MOE]
print('Confidence Interval =', confidence_interval)

t-statistic = 4.114705290861751
p-value = 3.942941513645935e-05
Margin of Error = 0.015258417380692
Confidence Interval = [0.016774437474368575, 0.04729127223575258]


In [25]:
# bootstrapping
permutation_replicates = np.empty(100000)

for i in range(len(permutation_replicates)):
    permutation_samples = np.random.permutation(np.concatenate((w_c, b_c)))
    
    white_perm = permutation_samples[:len(w_c)]
    black_perm = permutation_samples[len(w_c):]
    
    permutation_replicates[i] = np.abs(np.mean(white_perm) - np.mean(black_perm))

p = np.sum(permutation_replicates > diff_means) / len(permutation_replicates)
print('p-value =', p)

p-value = 0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

**#4. Write a story describing the statistical significance in the context or the original problem.**

Racial discrimination is often a problem encountered everyday in society, and more specifically in business. From our data, it is evident that race has a statisitcal significant impact on whether an applicant is called back or not, with white sounding names having a much greater callback rate than blank sounding names from their resumes. The p-values calculated from the tests are both very low, and much lower than the .05 threshold required. Based on this, we can reject the null hypothesis and conclude race impacts whether or not a resume will be called back.


**#5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?** 

The analysis does not conclude that race is the most important factor in determind a callback, even if it is a significant one. There is not enough other data to make that claim. In order to attempt to prove this, we would likely need more data and conduct more tests on what determines whether or not a resume is called back, including many other factors such as gender, age, background, work history, whether or not the job requirements are met, and so on.